In [1]:
import pickle

In [10]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import sklearn.metrics as metrics

In [5]:
with open('X_train_mecab.pkl', 'rb') as file:    
    X_train = pickle.load(file)
with open('X_test_mecab.pkl', 'rb') as file:    
    X_test = pickle.load(file)
with open('y_test_mecab.pkl', 'rb') as file:    
    y_test = pickle.load(file)
with open('y_train_mecab.pkl', 'rb') as file:   
    y_train = pickle.load(file)

In [12]:
model = Sequential()
model.add(layers.Embedding(27913, 100))
model.add(layers.SimpleRNN(units = 32,
                         activation = 'relu',
                         return_sequences = False))
model.add(layers.Dense(1, activation = 'sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         2791300   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                4256      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 2,795,589
Trainable params: 2,795,589
Non-trainable params: 0
_________________________________________________________________


In [18]:
model = Sequential()
model.add(layers.Embedding(27913, 150))
model.add(layers.GRU(units = 64,
                         activation = 'relu',
                         return_sequences = False))
model.add(layers.Dense(1, activation = 'sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 150)         4186950   
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                41472     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 4,228,487
Trainable params: 4,228,487
Non-trainable params: 0
_________________________________________________________________


In [19]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('best_model_GRU.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [20]:
model.compile(loss='binary_crossentropy', metrics=['acc'], optimizer = 'adam')
history = model.fit(X_train, y_train, epochs=10000, callbacks=[es, mc], batch_size=10, validation_split=0.2)

Epoch 1/10000
11909/11909 [==============================] - ETA: 0s - loss: 0.6384 - acc: 0.9269
Epoch 00001: val_acc improved from -inf to 0.94794, saving model to best_model_GRU.h5
11909/11909 [==============================] - 596s 50ms/step - loss: 0.6384 - acc: 0.9269 - val_loss: 0.1326 - val_acc: 0.9479
Epoch 2/10000
11908/11909 [============================>.] - ETA: 0s - loss: 0.1036 - acc: 0.9600
Epoch 00002: val_acc did not improve from 0.94794
11909/11909 [==============================] - 595s 50ms/step - loss: 0.1036 - acc: 0.9600 - val_loss: 0.1389 - val_acc: 0.9447
Epoch 3/10000
 7420/11909 [=================>............] - ETA: 3:38 - loss: 0.0734 - acc: 0.9723

KeyboardInterrupt: 

In [6]:
loaded_model = load_model('best_model.h5') 
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

1170/1170 [==============================] - 2s 2ms/step - loss: 0.1099 - acc: 0.9583

 테스트 정확도: 0.9583


In [29]:
loaded_model = load_model('best_model_mecab_simRNN.h5') 
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

1170/1170 [==============================] - 2s 2ms/step - loss: 0.1099 - acc: 0.9583

 테스트 정확도: 0.9583


In [31]:
from sklearn.metrics import confusion_matrix
pred = loaded_model.predict(X_test)
results = confusion_matrix(y_true=y_test, y_pred=pred.round())

print(results)

[[17337   565]
 [  997 18530]]


In [32]:
y_test_decode = []
for i in range(len(y_test)):
    if y_test[i] >= 0.5:
        y_test_decode.append(1)
    else:
        y_test_decode.append(0)

pred = loaded_model.predict(X_test)
pred_decode = []
for i in range(len(pred)):
    if pred[i] >= 0.5:
        pred_decode.append(1)
    else:
        pred_decode.append(0)


In [33]:
print('f1 : ',metrics.f1_score(y_test_decode, pred_decode, average='macro'))
print('recall : ', metrics.recall_score(y_test_decode,pred_decode,average='macro'))
print('precision : ',metrics.precision_score(y_test_decode,pred_decode,average='macro'))
print('accuracy', metrics.accuracy_score(y_test_decode, pred_decode,))

f1 :  0.958225213100645
recall :  0.9586908852065572
precision :  0.9580156308248793
accuracy 0.9582676534238158


#### mecab + simRNN
[17337   565]
<br>
[  997 18530]
 
f1 :  0.958225213100645
<br>
recall :  0.9586908852065572
<br>
precision :  0.9580156308248793
<br>
accuracy 0.9582676534238158